# Create meeting minutes from an Audio file

In [1]:
# Imports
import os
import requests
import warnings
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI
import ollama

In [2]:
# Constants
AUDIO_MODEL = "whisper-1"

In [3]:
# mp3 path
audio_filename = "./resources/audio_sample.mp3"

In [4]:
# Disable SSL warnings (optional but not recommended in production)
warnings.filterwarnings("ignore")

In [5]:
# Load OpenAI key from environment
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)

In [6]:
# Transcribe audio
api_url = "https://api.openai.com/v1/audio/transcriptions"
with open(audio_filename, "rb") as audio_file:
    files = {"file": audio_file}
    data = {"model": AUDIO_MODEL, "response_format": "text"}
    headers = {"Authorization": f"Bearer {openai_api_key}"}
    response = requests.post(api_url, files=files, data=data, headers=headers, verify=False)
transcription = response.text
print(transcription)

In [7]:
# Generate meeting minutes from transcription
system_message = "You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, in markdown."
user_prompt = f"Below is an extract transcript from a conversation. Please write minutes in markdown, including a summary with any relevant discussion points;\n{transcription}"
response = ollama.chat(
    model="llama3.2",
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt}
    ]
)
llm_response = response["message"]["content"]
display(Markdown(llm_response))